# Advanced usage of HiPS and MOCs to explore complex regions of interest

Stefania Amodea¹, Matthieu Baumann¹, Thomas Boch¹, Caroline Bot¹, Katarina A. Lutz¹, Manon Marchand¹. 

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

Thomas Boch and Caroline Bot wrote the original version of this tutorial, available on the [EURO-VO tutorials page](http://www.euro-vo.org/?q=science/scientific-tutorials). It was presented at the workshop "[Detecting the unexpected, Discovery in the Era of Astronomically Big Data](https://www.lsstcorporation.org/node/107)". The version here is an adaptation to jupyter notebooks by the Strasbourg astronomical Data Center ([CDS](https://cdsweb.u-strasbg.fr/)) team.

*** 

## Introduction

This tutorial demonstrates an advanced usage of Hierarchical Progressive Surveys ([HiPS](https://ui.adsabs.harvard.edu/abs/2017ivoa.spec.0519F/abstract)) and Multi-Order Coverage ([MOC](https://www.ivoa.net/documents/MOC/)) maps. Wis this tutorial, you will: 

1. learn how to find fits files
2. build a map of the coverage of all the fits files 
3. select only the parts of this map that correspond to low-extinction regions
4. retrieve objects from large catalogs inside the obtained map -- that has a non-trivial shape and non-necessarily-connected regions
5. combine catalogs to visualize a color-color diagram

In [ ]:
# General python packages
from pathlib import Path
import numpy as np

# For vizualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# Packages specific to astronomy tasks
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky
import astropy.units as u
import mocpy
import pyvo
import healpy as hp
from astroquery.vizier import Vizier


## Step 1: Finding the images

We want to find all Short-Red images in the Macquarie/AAO/Strasbourg Hα Planetary Galactic catalog ([MASH](https://vizier.cfa.harvard.edu/vizier/MASH/index.htx)) using the VizieR associated data service.

<img src=Data/images/logos/vizier.svg alt="vizier's logo" style="height:100px; display: inline-block;"/>

The [VizieR](https://vizier.cds.unistra.fr/) service at CDS inventories astronomical catalogs published in the literature. Some of these catalogs contain data associated with publications and the tables therein. This data can be browsed and explored through the VizieR-associated data service, linked to the traditional VizieR table service. Here we look for images associated with the MASH catalog of planetary nebulae ([Parker *et. al.* 2006-2008](https://ui.adsabs.harvard.edu/abs/2006yCat.5127....0P/abstract)). The MASH fits files are cut-outs extracted from a larger Hα and Short Red survey to constitute a set of regions of interest around planetary nebulae. 

To find VizieR-associated data, we use the Table Access Protocol (TAP) with the VizieR endpoint. Through the VizieR TAP endpoint, we can search for tables, their content, and information on associated data. 

First, we search for the MASH catalog:

In [ ]:
# give the address of the service, you can also directly visit the website
tap_vizier = pyvo.dal.TAPService(
    'https://tapvizier.cds.unistra.fr/TAPVizieR/tap')

# a query that searches for all tables with the words MASH and Parker in their description
query = '''
        SELECT  *  FROM tables 
        WHERE description LIKE '%MASH%Parker%'
        '''

mash_catalogues = tap_vizier.search(query).to_table()
mash_catalogues


In this tutorial, we are interested in the tables belonging to the catalogs `V/127A`. This includes tables `V/127A/mash1` and `V/127A/mash2`. To have a look at the content of these tables, we do the following:

In [ ]:
query = '''
        SELECT TOP 5 * FROM \"V/127A/mash1\" 
        '''
mash1_head = tap_vizier.search(query).to_table()
mash1_head

As you can see, the last column of this table is called `AssocData` and contains the entry `fits`. If you look at this table on the VizieR web interface, you can download the associated fits file. Within this notebook, we query the `obscore` database to get the URLs to the fits file. Using the `astropy.io.fits` module, we can open the fits files from their URLs.

In [ ]:
obs_tap_vizier = pyvo.dal.TAPService('https://cdsarc.cds.unistra.fr/saadavizier.tap/tap')
query = """
        SELECT TOP 5  *  FROM obscore 
        WHERE obs_collection='V/127A'   
        """
mash_fits = obs_tap_vizier.search(query).to_table()
mash_fits

As you can see, the result from this query provides information on the fits files associated with the MASH catalogs. In particular, the column `access_url` provides the location of the data. To get the first image we could do:

`image = fits.open(mash_fits['access_url'][0])`

and then work on the image, plot it or save it to our machine. However, downloading all the data takes quite some time. For this tutorial, **we prepared a subsample of 335 of these Short Red images that will run promptly** but we encourage you to try accessing the full Short Red sample on your own later. The subsample is available either at http://astro.u-strasbg.fr/~bot/BochBot.tar.gz or in the Data Folder of this repository. If you run this tutorial on Binder, you do not need to download anything. 

## Step 2: Create a MOC of the MASH images

The multi-order coverage (MOC) map of a set of images represents their sky coverage. MOCs can describe arbitrary zones in the sky which do not need to be connected. You'll see that the union or intersection of two MOCs requires few time and computational effort. Catalogs can also be filtered by MOCs. 

Here we want to use the fits files just downloaded to create a MOC map corresponding to the coverage of the MASH images.

### Organising data

In [ ]:
# Where to find fits images downloaded from the archive above
datadir = Path('Data/MASH_Sample/') 
datadir.mkdir(parents=True, exist_ok=True)
# Make new directory where to save MOCs
outdir = Path('Data/Result_moc/')        
outdir.mkdir(parents=True, exist_ok=True)

In most cases, we could ignore the next cell.  However, some possible deprecated keywords in the fits header would hamper the MOC creation and would cause errors in the underlying `astropy.wcs.WCS` module. This is why we rewrite the headers of the fits files so that they only contain the useful keywords needed to define the coordinate frame correctly before using `mocpy`. 

In [ ]:
# Get a list of all fits files in datadir
mash_file_list = datadir.glob('*_sr.fits')

# Modify the header so that the MOC creation works smoothly
for file_name in mash_file_list:
    ima = fits.open(file_name) 
    hdr = ima[0].header
    needed_keywords = ['SIMPLE','BITPIX', 'NAXIS','EXTEND',                     # basic fits keywords 
                       'NAXIS1', 'NAXIS2', 'RADECSYS',                          # essential WCS keywords
                       'CD1_1','CD1_2','CD2_1','CD2_2',                         
                       'CTYPE1', 'CUNIT1', 'CDELT1', 'CRPIX1', 'CRVAL1', 
                       'CTYPE2', 'CUNIT2', 'CDELT2', 'CRPIX2', 'CRVAL2',
                       'EQUINOX', 'BLANK'                                       # used additionally by mocpy
                        ]
    new_hdr = fits.Header()
    for keyword in needed_keywords:
        try:
            new_hdr[keyword] = ima[0].header[keyword]
        except KeyError:
            continue
    new_hdr['RADESYS'] = new_hdr['RADECSYS']
    fits.writeto(outdir / Path(file_name.stem + '_modified.fits'), 
                 ima[0].data, new_hdr, output_verify='fix', overwrite=True)
    ima.close()

### Create the MOC

Here we can create the MOC of the MASH images with the `MOCPy` module. 

In [ ]:
# Get a list of the updated files and create the MOC    
mash_file_list = outdir.glob('*_sr_modified.fits')
moc_mash = mocpy.MOC.from_fits_images(mash_file_list, max_norder=15)
moc_mash.write(outdir / 'mash_moc.fits', overwrite=True)

### Plot the MOC

We'll use the MOCpy `World2ScreenMPL` class.

In [ ]:
fig_centre = SkyCoord('21:49:00', '-02:30:00', 
                      unit=(u.hourangle, u.deg), frame='galactic')

fig = plt.figure(figsize=(10, 8))
with mocpy.World2ScreenMPL(fig, fov=2.5 * u.deg,
                           center=fig_centre,
                           coordsys="galactic", 
                           rotation=Angle(0, u.degree),
                           projection="AIT") as wcs:
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    moc_mash.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color='purple')
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    lon, lat = ax.coords[0], ax.coords[1]
    lon.set_major_formatter('hh:mm:ss')
    lat.set_major_formatter('dd:mm')
    lon.set_ticklabel(exclude_overlapping=True)
    lat.set_ticklabel(exclude_overlapping=True)

This figure only shows a small region on the Sky, but you can see how the MOC has arbitrary shapes and not all regions are connected.

## Step 3: Load an archival extinction map and create the MOC of the low extinction regions

Different works (e.g. [Schlegel et al. 1998](https://iopscience.iop.org/article/10.1086/305772), [Schlafly  & Finkbeiner 2011](https://ui.adsabs.harvard.edu/abs/2011AAS...21831803S/abstract), [Green et al. 2015](https://iopscience.iop.org/article/10.1088/0004-637X/810/1/25)...) have created extinction maps of the sky that are publicly available. Some of these maps are all-sky maps, while others have higher resolutions, or come from different methods... They can be found in HEALPix format (among others) on the Legacy Archive for Microwave Background Data Analysis ([LAMBDA](https://lambda.gsfc.nasa.gov/)) website or on the Analysis Center for Extended Data ([CADE](http://cade.irap.omp.eu/dokuwiki/doku.php?id=start)) website. 

For this tutorial, we will download the well-known all-sky extinction map from Schlegel *et al.* from the LAMBDA  website and define the low extinction area for which $0 < E(B-V) < 0.5$ as a MOC. It has an [information page](https://lambda.gsfc.nasa.gov/product/foreground/fg_sfd_info.html).

The map is available here: https://lambda.gsfc.nasa.gov/data/foregrounds/SFD/lambda_sfd_ebv.fits and we save it to disc. 

In [ ]:
ext_map = fits.open('https://lambda.gsfc.nasa.gov/data/foregrounds/SFD/' + 
                    'lambda_sfd_ebv.fits')
ext_map.writeto(outdir / 'Schlegel_extinction_map.fits', overwrite=True)
ext_map.close()

We are only interested in regions with low extinction. So we aim to get a MOC of all regions where the extinction values from the Schlegel *et al.* map are between 0 and 0.5mag. The extinction map we got from the NASA webpage is in the HEALPix format. This is an efficient presentation of all-sky maps. The HEALPix tesselation is also used by MOCs. So to get the MOC from the extinction map, we do the following. 

First, we check the coordinate system in the map header. We will need to convert to equatorial coordinates, change the projection of the map, and set the order (*i.e.* resolution) of the map.

In [ ]:
# open the downloaded FITS file
hdul = fits.open(outdir / 'Schlegel_extinction_map.fits')  
hdr = hdul[0].header
hdr

In [ ]:
nside = hdul[0].header['NSIDE']
norder = hdul[0].header['RESOLUTN']

The header allows to see that this map is in galactic coordinates. We will need to convert this into equatorial coordinates to compare with our other maps.

To do so, we'll use the function `rotate_map_pixel` of healpy that only accepts a ring-ordered map. This is why we load the map in a ring order, apply the coordinate change, and then transfer back to the nested ordering. 

In [ ]:
# Reads the map in the ringed Healpix ordering
extinction_map = hp.read_map('Data/Schlegel_extinction_map.fits', nest=False)
# Define a rotation from galactic to equatorial coordinates
r = hp.Rotator(coord=['G','C'])
# Do the coordinate rotation
extinction_map_equatorial_ring = r.rotate_map_pixel(extinction_map)
# Switch back to nested ordering
extinction_map_equatorial = hp.reorder(extinction_map_equatorial_ring, r2n=True) #reorder from RING to NESTED

In [ ]:
plt.hist(extinction_map_equatorial, bins=100);
print(f"""Min extinction: {extinction_map_equatorial.min()},
Max extinction: {extinction_map_equatorial.max()}
""")

Next we declare which pixel we want to use, let's take all pixels with an extinction lower than 0.5:

In [ ]:
indexes_pixels_to_keep = np.where(extinction_map_equatorial<0.5)[0]

# without the "[0]", np.where returns a tuple where the first element is the array of indices
# we need the array to pass to the MOC creator

Since we brought the extinction map in the right shape, the indexes of the pixels we want to keep are the indexes of the interesting Healpix cells. Thus the MOC is created by:

In [ ]:
moc_low_extinction = mocpy.MOC.from_healpix_cells(indexes_pixels_to_keep,
                                           np.full((len(indexes_pixels_to_keep,)), norder))
moc_low_extinction.save(str(outdir / 'low_extinction_moc.fits'), format="fits", overwrite=True)


## Step 4:  Find out which regions are covered by the MASH short-red images in the low extinction regions defined above

To find out the sky regions of the MASH sample that are at low extinction, we build the intersection of the two MOCs.

In [ ]:
moc_intersection = moc_low_extinction.intersection(moc_mash)
# Once the intersection is bluit, we can for example print the sky fraction :
print(f'The intersection of the two MOCs covers {round(moc_intersection.sky_fraction * 100, 4)}% of the sky')

Now we can visualize the coverage of the two MOCs and their intersection. The grey area is where the extinction is low. The blue one is the MASH coverage. The tiny red dots show the MASH coverage in low extinction regions. 

In [ ]:
fig = plt.figure(111, figsize=(10, 8))
with mocpy.World2ScreenMPL(fig, fov=200 * u.deg,
                           center=SkyCoord(200, -20, unit='deg', frame='icrs'),
                           coordsys="icrs",
                           rotation=Angle(0, u.degree),
                           projection="AIT") as wcs:
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    moc_low_extinction.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="grey",label='low extinction')
    moc_mash.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="dodgerblue", label='MASH coverage')
    moc_intersection.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="crimson", label='MASH in low extinction reg.')
    # Sets labels
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    # Sets ticks
    lon, lat = ax.coords[0], ax.coords[1]
    lon.set_major_formatter('hh:mm:ss')
    lat.set_major_formatter('dd:mm')
    lon.set_ticklabel(exclude_overlapping=True)
    lat.set_ticklabel(exclude_overlapping=True)
    lon.set_ticks(spacing=2 * u.hourangle)
plt.legend()
plt.show()

## Step 5: Query the 2MASS and Gaia Catalogues by MOC

Without the usage of MOC, querying for sources in the low extinction regions covered by the MASH subsample would be tedious or even impossible. Indeed, one would need to load the whole catalog and make selections which would not be possible given the size of some catalogs. Alternatively, one would need to query the catalog field by field, which would take time and several queries. Instead, here we will use the power of MOC files to query large catalogs directly in the covered regions only. We will use coverages of the low extinction and MASH regions to query for sources from the Gaia and 2MASS surveys in these highly non-continuous and non-trivial shape areas.

First, let's see which Gaia and 2MASS catalogs are available on VizieR. We could, as above, use the TAP endpoint of VizieR. But we show below the `Vizier` module in the `astroquery` package.

In [ ]:
catalog_list_twomass = Vizier.find_catalogs('Cutri')
for k, v in catalog_list_twomass.items():
    print(k, ': ', v.description)

In [ ]:
catalog_list_gaia = Vizier.find_catalogs('Gaia DR2', max_catalogs=1000)
for k, v in catalog_list_gaia.items():
    print(k, ': ', v.description)

For 2MASS we will want to use `II/246 :  2MASS All-Sky Catalog of Point Sources (Cutri+ 2003)` and for Gaia `I/345 :  Gaia DR2 (Gaia Collaboration, 2018)`. Before we query the full two tables we only look at a few sources for each table to understand which columns are available. The query below will give us 50 sources each -- the default for the `get_catalogs` method.

In [ ]:
test_twomass = Vizier.get_catalogs('II/246')
print(test_twomass)
test_twomass[0]

In [ ]:
test_gaia = Vizier.get_catalogs('I/345')
print(test_gaia)
test_gaia[0]

As you will see below, we only need coordinates, 2MASS photometry in the H and K band, and Gaia photometry in the Gaia G band. So we'll query the tables `II/246/out` for 2MASS and `I/345/gaia2` for Gaia DR2:

In [ ]:
twomass = moc_intersection.query_vizier_table('II/246/out', max_rows=20000)
twomass

In [ ]:
gaia = moc_intersection.query_vizier_table('I/345/gaia2', max_rows=20000)
gaia

## Step 6: Cross-match Gaia and WISE sources in all fields

We now want to find sources in the selected region (observed in the MASH regions of interest and at low extinction) that are common to the Wide Infrared Survey Explorer ([WISE](https://irsa.ipac.caltech.edu/Missions/wise.html)) and Gaia catalogs. To do so, we will perform a cross-match of the Gaia and WISE catalogs. Alternatively, we could use the CDS XMatch service via the corresponding `astroquery` module.

To do so, let's first inspect the `match_coordinates_sky` function from `astropy.coordinates`. 

In [ ]:
help(match_coordinates_sky)

In [ ]:
# We generate the coordinates in the appropriate format
twomass_coord = SkyCoord(ra=twomass['RAJ2000'], dec=twomass['DEJ2000'], unit=u.deg)
gaia_coord = SkyCoord(ra=gaia['ra_epoch2000'], 
                      dec=gaia['dec_epoch2000'], unit=u.deg)

index, separation_2d, _ = match_coordinates_sky(twomass_coord, gaia_coord)

In [ ]:
# Decide the maximum separation between objects to be considered acceptable matches
max_separation = 1.0 * u.arcsec
# Apply constraint on the two catalogs
sep_constraint = separation_2d < max_separation
twomass_matches = twomass[sep_constraint]
gaia_matches = gaia[index[sep_constraint]]
# Select only interesting columns from twomass_matches
match_catalog = twomass_matches['_2MASS', 'RAJ2000', 'DEJ2000', 'Hmag', 'Kmag']
# Add column G magnitude from gaia
match_catalog['Gmag'] = gaia_matches['phot_g_mean_mag']
match_catalog

## Step 7: Build a color-color diagram

We now use the data we got from the cross-match to get a WISE/Gaia color-color diagram for all the sources in the low extinction sky regions covered by the MASH survey:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(match_catalog['Hmag'] - match_catalog['Kmag'], 
        match_catalog['Gmag'] - match_catalog['Hmag'], linestyle='', marker='.')
ax.set_xlabel('H - Ks [mag]',fontsize=16)
ax.set_ylabel('G - H [mag]',fontsize=16)
plt.show()